In [8]:
import numpy as np
import json
import matplotlib
import matplotlib.pyplot as plt
import pandas as pd

import xobjects as xo
import xtrack as xt
import xpart as xp
import xcoll as xc

import xcoll_plotting as xplt


context = xo.ContextCpu() 

# Test line generation

In [11]:
#line = xt.Line.from_json('../input_files/flat_top_b2.json')
#line = xt.Line.from_json('TEST_v3.json')
line = xt.Line.from_json('TEST_nooct.json')
line.particle_ref = xp.Particles(p0c=6800e9, q0=1, mass0=xp.PROTON_MASS_EV)

Done loading line from dict.           


In [12]:
line.twiss()

Compiling ContextCpu kernels...
Done compiling ContextCpu kernels.


TwissTable: 103384 rows, 35 cols

In [1]:
! cmp --silent TEST_v2.json TEST_nooct.json && echo '### SUCCESS: Files Are Identical! ###' || echo '### WARNING: Files Are Different! ###'

### WARNING: Files Are Different! ###


In [3]:
! cmp --silent TEST_v2.json TEST_v3.json && echo '### SUCCESS: Files Are Identical! ###' || echo '### WARNING: Files Are Different! ###'

### SUCCESS: Files Are Identical! ###


# Test closed orbit after aperture correction

In [ ]:
def find_axis_intercepts(x_coords, y_coords):
    x_intercepts = []
    y_intercepts = []

    for i in range(len(x_coords)):
        x1, y1 = x_coords[i], y_coords[i]
        x2, y2 = x_coords[(i + 1) % len(x_coords)], y_coords[(i + 1) % len(y_coords)]

        if x1 == x2:
        # Vertical line, no y-intercept
            y_intercept = 0.0 if x1 == x2 == 0.0 else None
        else:
            slope = (y2 - y1) / (x2 - x1)
            y_intercept = y1 - (slope * x1)

        if y1 == y2:
        # Horizontal line, no x-intercept
            x_intercept = 0.0 if y1 == y2 == 0.0 else None
        else:
            slope = (x2 - x1) / (y2 - y1)
            x_intercept = x1 - (slope * y1)

        # Check if the x-intercept is within the range of x1 and x2
        if x_intercept is not None and (x1 <= x_intercept <= x2 or x2 <= x_intercept <= x1):
            x_intercepts.append(x_intercept)

        # Check if the y-intercept is within the range of y1 and y2
        if y_intercept is not None and (y1 <= y_intercept <= y2 or y2 <= y_intercept <= y1):
            y_intercepts.append(y_intercept)

    return x_intercepts, y_intercepts



def find_bad_offset_apertures(line):
    aperture_offsets = {}
    for name, element in line.element_dict.items():
        if 'offset' in name and element.__class__.__name__.startswith('XYShift'):
            aper_name = name.split('_offset')[0]
            aperture_offsets[aper_name] = (element.dx, element.dy)

    bad_apers = {}
    print('vmabc.4l2.b.b2_aper' in aperture_offsets.keys())
    for ap_name, offset in aperture_offsets.items():
        aperture_el = line.element_dict[ap_name]

        cname= aperture_el.__class__.__name__
        ap_dict = aperture_el.to_dict()

        if cname == 'LimitEllipse':
            x_min = -ap_dict['a']
            x_max = ap_dict['a']
            y_min = -ap_dict['b']
            y_max = ap_dict['b']
        elif cname == 'LimitRect':
            x_min = ap_dict['min_x']
            x_max = ap_dict['max_x']
            y_min = ap_dict['min_y']
            y_max = ap_dict['max_y']
        elif cname == 'LimitRectEllipse':
            x_min = -ap_dict['max_x']
            x_max = ap_dict['max_x']
            y_min = -ap_dict['max_y']
            y_max = ap_dict['max_y']
        elif cname == 'LimitRacetrack':
            x_min = ap_dict['min_x']
            x_max = ap_dict['max_x']
            y_min = ap_dict['min_y']
            y_max = ap_dict['max_y']
        elif cname == 'LimitPolygon':
            x_intercepts, y_intercepts = find_axis_intercepts(ap_dict['x_vertices'],
                                                            ap_dict['y_vertices'])
            x_min = min(x_intercepts)
            x_max = max(x_intercepts)
            y_min = min(y_intercepts)
            y_max = max(y_intercepts)

        tolerance = 5e-3
        """if (x_max - offset[0] < tolerance 
            or -x_min + offset[0] < tolerance 
            or y_max - offset[1] < tolerance 
            or -y_min + offset[1] < tolerance):"""
        if (offset[0] -x_max > tolerance 
            or  -offset[0] + x_min > tolerance 
            or  offset[1] - y_max > tolerance 
            or  -offset[1] + y_min > tolerance ):
                bad_apers[ap_name] = (x_min, x_max, y_min, y_max, offset[0], offset[1])

    return bad_apers

In [ ]:
config_file = "config_sim.yaml"
with open(config_file, 'r') as stream:
    config_dict = yaml.safe_load(stream)



sub_dict = config_dict['run']
file_dict = config_dict['input_files']


context = xo.ContextCpu(omp_num_threads='auto')

# On a modern CPU, we get ~5000 particle*turns/s
# So this script should take around half an hour
beam          = sub_dict['beam']
plane         = sub_dict['plane']

num_turns     = sub_dict['turns']
num_particles = sub_dict['nparticles']
engine        = sub_dict['engine']


path_out = Path.cwd() / 'Outputdata'

if not path_out.exists():
    os.makedirs(path_out)


# Load from json
line = xt.Line.from_json(file_dict[f'line_b{beam}'])


#line.element_dict('vmabc.4l2.b2_aper').max_x=100

#line.particle_ref = xp.Particles(p0c=6800e9, q0=1, mass0=xp.PROTON_MASS_EV)
#tw=line.twiss()

end_s = line.get_length()


TCCS_loc = end_s - 6673.7 #6775
TCCP_loc = end_s - 6653.3 #6655
TARGET_loc = end_s - (6653.3 + 0.07/2 +0.005/2)


line.insert_element(at_s=TCCS_loc, element=xt.Marker(), name='tccs.3.b2')
line.insert_element(at_s=TCCS_loc, element=xt.LimitEllipse(a_squ=0.0016, b_squ=0.0016, a_b_squ=2.56e-06), name='tccs.3.b2_aper')
line.insert_element(at_s=TCCP_loc, element=xt.Marker(), name='tccp.3.b2')
line.insert_element(at_s=TCCP_loc, element=xt.LimitEllipse(a_squ=0.0016, b_squ=0.0016, a_b_squ=2.56e-06), name='tccp.3.b2_aper')
line.insert_element(at_s=TARGET_loc, element=xt.Marker(), name='target.3.b2')
line.insert_element(at_s=TARGET_loc, element=xt.LimitEllipse(a_squ=0.0016, b_squ=0.0016, a_b_squ=2.56e-06), name='target.3.b2_aper')

In [ ]:
bad_aper = find_bad_offset_apertures(line)
print('!! Bad apertures : ', bad_aper)

In [ ]:
print('Replace bad apertures with Marker')
for name in bad_aper.keys():
    line.element_dict[name] = xt.Marker()
    print(line.element_dict[name])


for name in bad_aper.keys():
    print(name, line.element_dict[name])

In [ ]:

# Aperture model check
print('\nAperture model check on imported model:')
df_imported = line.check_aperture()
assert not np.any(df_imported.has_aperture_problem)




# Initialise collmanager
coll_manager = xc.CollimatorManager.from_yaml(file_dict['collimators'], line=line, beam=beam, _context=context, ignore_crystals=False)
#print(coll_manager.collimator_names)

# Install collimators into line
if engine == 'everest':
    coll_manager.install_everest_collimators(verbose=True)
else:
    raise ValueError(f"Unknown scattering engine {engine}!")

In [ ]:
line[name].align_angle

In [ ]:
# Aperture model check
print('\nAperture model check after introducing collimators:')
df_with_coll = line.check_aperture()
assert not np.any(df_with_coll.has_aperture_problem)

    
# Build the tracker
coll_manager.build_tracker()


# Set the collimator openings based on the colldb,
# or manually override with the option gaps={collname: gap}
coll_manager.set_openings()

# Aperture model check
print('\nAperture model check after introducing collimators:')
df_with_coll = line.check_aperture()
#assert not np.any(df_with_coll.has_aperture_problem))

### Test angle crystal 

In [ ]:
line[name].align_angle
line[name].align_angle = line[name].align_angle+2

### Simulation


In [ ]:
    # Generate initial pencil distribution on horizontal collimator
    tcp  = f"tcp.{'c' if plane=='H' else 'd'}6{'l' if beam=='1' else 'r'}7.b{beam}"
    part = coll_manager.generate_pencil_on_collimator(tcp, num_particles=num_particles)


    # Optimise the line
    #line.optimize_for_tracking()
    idx = line.element_names.index(tcp)
    part.at_element = idx
    part.start_tracking_at_element = idx


    # Track
    coll_manager.enable_scattering()
    line.track(part, num_turns=num_turns, time=True)
    coll_manager.disable_scattering()
    print(f"Done tracking in {line.time_last_track:.1f}s.")

# Line Analysis

In [ ]:
stop = line.element_names.index('tcla.a5l3.b2')
start = line.element_names.index('tcsg.5r3.b2')

In [ ]:
for i in range(start-400, stop+400):
    if (line.element_names[i].endswith('b2') and type(line[i]) !=  xt.beam_elements.elements.Marker) or line.element_names[i].startswith('ip'):
        print(line.element_names[i], '\t\t', end_s-line.get_s_elements()[i], '\t\t', line[i].__class__.__name__ )